# Imports

In [ ]:
import numpy as np
import pandas as pd
import os
import random
import shutil
import glob
import yaml
import torch
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import cv2

from PIL import Image
from matplotlib import pyplot as plt
from matplotlib import patches
from pathlib import Path
from sklearn.model_selection import train_test_split

# Install YOLOv8

In [ ]:
import ultralytics
from ultralytics import YOLO
ultralytics.checks()

# Create Directory

In [ ]:
data = "/kaggle/input/face-mask-detection"
image_directory = data + "/images"
annotation_directory = data + "/annotations"
annotations = list(Path(annotation_directory).glob(r'**/*{}'.format('xml')))

# Data Preprocessing

In [ ]:
class_id = {
    "with_mask" : 0,
    "mask_weared_incorrect" : 1,
    "without_mask" : 2
}

data_dict = {
    'filename': [],
    'label': [],
    'class_id': [],
    'width': [],
    'height': [],
    'bboxes': []
}
for annotation_path in annotations:
    tree = ET.parse(annotation_path)
    root = tree.getroot()
    filename = root.find('filename').text
    for obj in root.findall("object"):
        label = obj.find("name").text
        
        bbox = []
        # bndbox has xmin, ymin, xmax, ymax
        bndbox_tree = obj.find('bndbox')
        bbox.append(int(bndbox_tree.find('xmin').text))
        bbox.append(int(bndbox_tree.find('ymin').text))
        bbox.append(int(bndbox_tree.find('xmax').text))
        bbox.append(int(bndbox_tree.find('ymax').text))
        size = root.find('size')
        
        data_dict['filename'].append(filename)
        data_dict['width'].append(int(size.find('width').text))
        data_dict['height'].append(int(size.find('height').text))
        data_dict['label'].append(label)
        data_dict['class_id'].append(class_id[label])
        data_dict['bboxes'].append(bbox)

df_data = pd.DataFrame(data_dict)

df_data.head()

In [ ]:
df_data.isna().sum()

In [ ]:
df_data.info()

In [ ]:
df_data.label.unique()

In [ ]:
print(f"Total 'without_mask' labels: {sum(df_data.label == 'without_mask')}")
print(f"Total 'mask_weared_incorrect' labels: {sum(df_data.label == 'mask_weared_incorrect')}")
print(f"Total 'with_mask' labels: {sum(df_data.label == 'with_mask')}")

# Visualize Data

In [ ]:
def show_random_images_with_bbox(df):
    all_images = os.listdir(image_directory)
    random_image_filename = random.sample(all_images, 4)
    fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(10, 10))
    for i, filename in enumerate(random_image_filename):
        selected_df = df[df['filename'] == filename]
        
        image = Image.open(image_directory + '/' + filename)
        
        ax.flat[i].imshow(image)
        ax.flat[i].axis(False)
        
        image_bboxes = []
        for df_index in range(0, len(selected_df)):
            color = "g"
            if selected_df.iloc[df_index].class_id == 1: color = "y"
            elif selected_df.iloc[df_index].class_id == 2: color = "r"
            
            x_min, y_min, x_max, y_max = selected_df.iloc[df_index].bboxes
            
            rect = patches.Rectangle([x_min, y_min], x_max-x_min, y_max-y_min, 
                             linewidth=2, edgecolor=color, facecolor="none")
            ax.flat[i].add_patch(rect)
            
show_random_images_with_bbox(df_data)

In [ ]:
#Converting the pascal voc to yolo bbox before building the model
def pascal_voc_to_yolo_bbox(bbox_array, w, h):
    x_min, y_min, x_max, y_max = bbox_array
    
    x_center = ((x_max + x_min) / 2) / w
    y_center = ((y_max + y_min) / 2) / h
    
    width = (x_max - x_min) / w
    height = (y_max - y_min) / h
    
    return [x_center, y_center, width, height]

# Split Train, Val, Test images

Creating a new directory to store the images

In [ ]:
tr_path = "/kaggle/working/datasets/train"
val_path = "/kaggle/working/datasets/valid"
test_path = "/kaggle/working/datasets/test"

os.mkdir("/kaggle/working/datasets")
os.mkdir(tr_path)
os.mkdir(val_path)
os.mkdir(test_path)

Moving the images to their respective paths and create corresponding labels

In [ ]:
train, test = train_test_split(df_data.filename.unique(), test_size=0.2, random_state=23)
train, valid = train_test_split(train, test_size=0.15, random_state=23)

def copy_image_file(image_items, folder_name):
    for image in image_items:
            image_path = image_directory + "/" + image
            new_image_path = os.path.join(folder_name, image)
            shutil.copy(image_path, new_image_path)

def create_label_file(image_items, folder_name):
    for image in image_items:
        fileName = Path(image).stem
        df = df_data[df_data['filename'] == image]
        with open(folder_name + "/" + fileName +'.txt', 'w') as f:
            for i in range(0, len(df)):
                bbox = pascal_voc_to_yolo_bbox(df.iloc[i]['bboxes'], df.iloc[i]['width'], df.iloc[i]['height'])
                bbox_text = " ".join(map(str, bbox))
                txt = str(df.iloc[i]['class_id'])+ " " + bbox_text
                f.write(txt)
                if i != len(df) - 1:
                    f.write("\n")
                

copy_image_file(train, train_path)
copy_image_file(valid, valid_path)
copy_image_file(test, test_path)

create_label_file(train, train_path)
create_label_file(valid, valid_path)
create_label_file(test, test_path)

In [ ]:
def walk_through_dir(filepath):
    for dirpath, dirnames, filenames in os.walk(filepath):
        print(f"There are {len(dirnames)} directories and {len(glob.glob(filepath + '/*.png', recursive = True))} images in '{dirpath}'.")
    
walk_through_dir(train_path)
walk_through_dir(valid_path)  
walk_through_dir(test_path)

# Create YAML file

In [ ]:
classes = list(df_data.label.unique())
class_count = len(classes)
facemask_yaml = f"""
    train: train
    val: valid
    test: test
    nc: {class_count}
    names:
        0 : with_mask
        1 : mask_weared_incorrect
        2 : without_mask
    """

with open('facemask.yaml', 'w') as f:
    f.write(facemask_yaml)
    
%cat facemask.yaml

# Train

Building the model and setting up the requirments

In [ ]:
model = YOLO("yolov8n.pt") 
model.train(data="facemask.yaml", epochs=10)

In [ ]:
model.val(data="facemask.yaml")

In [ ]:
import matplotlib.pyplot as plt

# Class names and corresponding metrics
class_names = ['with_mask', 'mask_weared_incorrect', 'without_mask', 'all']
precision = [0.915, 0.793, 0.765, 0.824]
recall = [0.838, 0.286, 0.713, 0.612]
mAP50 = [0.91, 0.35, 0.721, 0.661]
mAP50_95 = [0.575, 0.274, 0.439, 0.429]

# Plot precision, recall, mAP50, mAP50-95
plt.figure(figsize=(10, 6))
plt.bar(class_names, precision, label='Precision', color='skyblue')
plt.bar(class_names, recall, label='Recall', color='lightcoral')
plt.bar(class_names, mAP50, label='mAP50', color='limegreen')
plt.bar(class_names, mAP50_95, label='mAP50-95', color='orange')
plt.xlabel('Class')
plt.ylabel('Value')
plt.title('YOLOv8 Evaluation Results')
plt.legend()
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Class names and corresponding metrics
class_names = ['with_mask', 'mask_weared_incorrect', 'without_mask', 'all']
precision = [0.915, 0.793, 0.765, 0.824]
recall = [0.838, 0.286, 0.713, 0.612]
mAP50 = [0.91, 0.35, 0.721, 0.661]
mAP50_95 = [0.575, 0.274, 0.439, 0.429]

# Set the bar width
bar_width = 0.2

# Position of bars on the x-axis
r1 = np.arange(len(class_names))
r2 = [x + bar_width for x in r1]
r3 = [x + bar_width for x in r2]
r4 = [x + bar_width for x in r3]

# Create the plot
plt.figure(figsize=(10, 6))
plt.bar(r1, precision, color='skyblue', width=bar_width, edgecolor='black', label='Precision')
plt.bar(r2, recall, color='lightcoral', width=bar_width, edgecolor='black', label='Recall')
plt.bar(r3, mAP50, color='limegreen', width=bar_width, edgecolor='black', label='mAP50')
plt.bar(r4, mAP50_95, color='orange', width=bar_width, edgecolor='black', label='mAP50-95')

# Add values above bars
for i, v in enumerate(precision):
    plt.text(i - bar_width / 2, v + 0.02, str(round(v, 3)), color='black', ha='center', fontsize=8)
for i, v in enumerate(recall):
    plt.text(i + bar_width / 2, v + 0.02, str(round(v, 3)), color='black', ha='center', fontsize=8)
for i, v in enumerate(mAP50):
    plt.text(i + bar_width * 1.5, v + 0.02, str(round(v, 3)), color='black', ha='center', fontsize=8)
for i, v in enumerate(mAP50_95):
    plt.text(i + bar_width * 2.5, v + 0.02, str(round(v, 3)), color='black', ha='center', fontsize=8)

# Customize the plot
plt.xlabel('Class')
plt.ylabel('Value')
plt.title('YOLOv8 Evaluation Results for 10 Epochs')
plt.xticks([r + 1.5 * bar_width for r in range(len(class_names))], class_names)
plt.legend(loc='upper left')
plt.grid(axis='y')

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
model1 = YOLO("yolov8n.pt") 
model1.train(data="facemask.yaml", epochs=25)

In [ ]:
model1.val(data="facemask.yaml")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Class names and corresponding metrics
class_names = ['with_mask', 'mask_weared_incorrect', 'without_mask', 'all']
precision = [0.945, 1.0, 0.822, 0.922]
recall = [0.856, 0.353, 0.695, 0.635]
mAP50 = [0.937, 0.603, 0.732, 0.757]
mAP50_95 = [0.601, 0.406, 0.434, 0.481]

# Set the bar width
bar_width = 0.2

# Position of bars on the x-axis
r1 = np.arange(len(class_names))
r2 = [x + bar_width for x in r1]
r3 = [x + bar_width for x in r2]
r4 = [x + bar_width for x in r3]

# Create the plot
plt.figure(figsize=(10, 6))
plt.bar(r1, precision, color='skyblue', width=bar_width, edgecolor='black', label='Precision')
plt.bar(r2, recall, color='lightcoral', width=bar_width, edgecolor='black', label='Recall')
plt.bar(r3, mAP50, color='limegreen', width=bar_width, edgecolor='black', label='mAP50')
plt.bar(r4, mAP50_95, color='orange', width=bar_width, edgecolor='black', label='mAP50-95')

# Add values above bars
for i, v in enumerate(precision):
    plt.text(i - bar_width / 2, v + 0.02, str(round(v, 3)), color='black', ha='center', fontsize=8)
for i, v in enumerate(recall):
    plt.text(i + bar_width / 2, v + 0.02, str(round(v, 3)), color='black', ha='center', fontsize=8)
for i, v in enumerate(mAP50):
    plt.text(i + bar_width * 1.5, v + 0.02, str(round(v, 3)), color='black', ha='center', fontsize=8)
for i, v in enumerate(mAP50_95):
    plt.text(i + bar_width * 2.5, v + 0.02, str(round(v, 3)), color='black', ha='center', fontsize=8)

# Customize the plot
plt.xlabel('Class')
plt.ylabel('Value')
plt.title('YOLOv8 Evaluation Results for 25 Epochs')
plt.xticks([r + 1.5 * bar_width for r in range(len(class_names))], class_names)
plt.legend(loc='upper left')
plt.grid(axis='y')

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
model2 = YOLO("yolov8n.pt") 
model2.train(data="facemask.yaml", epochs=40)

In [ ]:
model2.val(data="facemask.yaml")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Class names and corresponding metrics
class_names = ['with_mask', 'mask_weared_incorrect', 'without_mask', 'all']
precision = [0.932, 0.744, 0.837, 0.838]
recall = [0.886, 0.714, 0.754, 0.784]
mAP50 = [0.931, 0.7, 0.813, 0.815]
mAP50_95 = [0.619, 0.471, 0.477, 0.522]

# Set the bar width
bar_width = 0.2

# Position of bars on the x-axis
r1 = np.arange(len(class_names))
r2 = [x + bar_width for x in r1]
r3 = [x + bar_width for x in r2]
r4 = [x + bar_width for x in r3]

# Create the plot
plt.figure(figsize=(10, 6))
plt.bar(r1, precision, color='skyblue', width=bar_width, edgecolor='black', label='Precision')
plt.bar(r2, recall, color='lightcoral', width=bar_width, edgecolor='black', label='Recall')
plt.bar(r3, mAP50, color='limegreen', width=bar_width, edgecolor='black', label='mAP50')
plt.bar(r4, mAP50_95, color='orange', width=bar_width, edgecolor='black', label='mAP50-95')

# Add values above bars
for i, v in enumerate(precision):
    plt.text(i - bar_width / 2, v + 0.02, str(round(v, 3)), color='black', ha='center', fontsize=8)
for i, v in enumerate(recall):
    plt.text(i + bar_width / 2, v + 0.02, str(round(v, 3)), color='black', ha='center', fontsize=8)
for i, v in enumerate(mAP50):
    plt.text(i + bar_width * 1.5, v + 0.02, str(round(v, 3)), color='black', ha='center', fontsize=8)
for i, v in enumerate(mAP50_95):
    plt.text(i + bar_width * 2.5, v + 0.02, str(round(v, 3)), color='black', ha='center', fontsize=8)

# Customize the plot
plt.xlabel('Class')
plt.ylabel('Value')
plt.title('YOLOv8 Evaluation Results for 40 Epochs')
plt.xticks([r + 1.5 * bar_width for r in range(len(class_names))], class_names)
plt.legend(loc='upper left')
plt.grid(axis='y')

# Show the plot
plt.tight_layout()
plt.show()

# Validation

In [ ]:
confusion_matrix = Image.open("/kaggle/working/runs/detect/train/confusion_matrix.png")
plt.figure(figsize=(20,10))
plt.imshow(confusion_matrix)
plt.axis(False)
plt.show()

In [ ]:
val_label = Image.open("/kaggle/working/runs/detect/train/val_batch0_labels.jpg")
val_pred = Image.open("/kaggle/working/runs/detect/train/val_batch0_pred.jpg")

plt.figure(figsize=(20,10))
plt.imshow(val_label)
plt.title("Label")
plt.axis(False)
plt.show()

plt.figure(figsize=(20,10))
plt.imshow(val_pred)
plt.title("Prediction")
plt.axis(False)
plt.show()

# Predict

In [ ]:
model = YOLO(model="/kaggle/working/runs/detect/train/weights/best.pt")

In [ ]:
filenames = glob.glob(test_path+"/*.png", recursive=False)
test_image1 = cv2.imread(filenames[0])
test_image2 = cv2.imread(filenames[1])

results = model.predict([test_image1, test_image2], save=True, line_thickness=1)

In [ ]:
predicted_image = Image.open("runs/detect/predict/image0.jpg")
plt.figure(figsize=(10,10))
plt.imshow(predicted_image)
plt.title("Prediction")
plt.axis(False)
plt.show()

predicted_image = Image.open("runs/detect/predict/image1.jpg")
plt.figure(figsize=(10,10))
plt.imshow(predicted_image)
plt.title("Prediction")
plt.axis(False)
plt.show()